<a href="https://colab.research.google.com/github/LVaditya/BusinessCard-Reader/blob/main/BusinessCard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr 

In [ ]:
!pip install streamlit

In [ ]:
!pip install pyngrok

In [4]:
from pyngrok import ngrok

In [5]:
ngrok.set_auth_token('2L6Kz7Hvawi3QB7akHGUu8CRsfi_5JPuFqERA67quEKGFuvaU')

In [25]:
%%writefile app.py

# import all the packages needed

import streamlit as st 
from PIL import Image
import easyocr
import pandas as pd
import numpy as np
import re
import os
import sqlite3


conn = sqlite3.connect('Businesscard.db')
c = conn.cursor()


#-------Page Configuration--------

st.set_page_config(page_title="Business Card Reader",
                    page_icon=":black_small_square:",
                    layout="wide"
                    )

st.title("Business Card Reader")


#loading the image


def main():
 
  #file to be uploaded is taken as image_file

  image_file= st.file_uploader("upload the image of business card you want to extract data", type=["jpg","png","jpeg"])

  #-----------------image file read as binary file into image-----

  with open(os.path.join("tempDir",image_file.name),"wb") as f: 
    f.write(image_file.getbuffer()) 
  #image_file_path=os.path.abspath(image_file)
  #with open(image_file_path,'r') as f:
   # image = f.read()

#-------image file is read using OCR------
  @st.cache_data
  def load_model():
    reader=easyocr.Reader(['en'])
    return reader
  
  reader=load_model()

  #-----------------if the image is uploaded------

  if image_file is not None:

    img = Image.open(image_file)  #read image
    st.image(img, width=400)                 #display image

    with st.spinner("Please wait.... work in progress"):

      data = reader.readtext(np.array(img))       #load the data from image as data
      text_result=[]

      text_dict={'Name':[], 'Designation':[], 'Phone':[], 'Email':[], 'Website':[], 'City':[], 'State':[], 'Pincode':[]}
      for text in data:
        text_result.append(text[1])

      text_dict['Name'] = text_result[0]
      text_dict['Designation']= text_result[1]


      for text in text_result:
        if 'WWW'in text or 'www' in text or 'wwW' in text:
          text_dict['Website'] = text

        elif '@' in text: 
          text_dict['Email']= text

        elif '-' in text:
          text_dict['Phone'] = text
        
        else:
          l=list(map(str,text.split(" ")))
          for i in l:
            i=i.replace(",","")
            i=i.replace(";","")
            if i.isdigit():
              text_dict['Pincode'] = i
            elif i.lower() in ['tamilnadu', 'kerala','karnataka', 
            'maharashtra', 'delhi', 'andhra pradesh', 'telangana' ]:
              text_dict['State'] = i
            elif i.lower() in ['chennai', 'hydrabad', 'salem', 'erode', 'tirupur', 'coimbatore']:
              text_dict['City'] = i

      st.experimental_data_editor(data=text_dict)            #All collected data is displayed in table format

      
      #text_dict['Image']= image_file.name
      #-------------providing edit options in the GUI---------

  #if "button_clicked" not in st.session_state:
   # st.session_state.button_clicked = False

  #def callback():
   # st.session_state.button_clicked = True 

  #if(st.button("edit", on_click = callback) or
   #   st.session_state.button_clicked):
    #choice = st.select("select",['Name', 'Designation','Email','Phone','Website','city','State','Pincode'])
    #if choice== 'Name':
     # Text_dict['Name']=st.text_intput("Update Name")
          
      


     
      df=pd.DataFrame(text_dict, index=[0])    #converting into dataframe

#-----------to upload the data into sqlite------

    upload = st.button("upload data")

    if upload:
      try:
        df.to_sql('Card_values',conn,if_exists='replace',index=False)    # data appended into a table in database
       


      except:
        print("connection error")

    conn.commit()
    c.close()
    conn.close()

#----------if image not uploaded-----
    
  else:
    st.write("upload an image of business card")



  
if __name__=='__main__':
  main()

Overwriting app.py


In [26]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://9766-34-80-43-5.ngrok.io" -> "http://localhost:80"


In [28]:
import sqlite3
import pandas as pd

In [29]:
conn=sqlite3.connect('Businesscard.db')
c=conn.cursor()


In [12]:
c.execute('''select* from Card_values''')

In [13]:
query='''select * from Card_values'''

In [19]:
m= c.execute(query)

In [21]:
for x in m:
  print(x)

In [ ]:
with open('card_image.png','wb') as f:
  f.write(rec_data)

In [14]:
df_new = pd.read_sql(query,conn)

In [17]:
df_new

,Name,Designation,Phone,Email,Website,City,State,Pincode,Image
